# Uncertainties

```{autolink-concat}
```

In [ ]:
from __future__ import annotations

from collections import defaultdict

import jax.numpy as jnp
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import Latex, Markdown
from matplotlib import cm
from tensorwaves.function.sympy import create_parametrized_function
from tqdm.auto import tqdm

from polarization import formulate_polarization
from polarization.amplitude import DalitzPlotDecompositionBuilder
from polarization.data import create_data_transformer, generate_meshgrid_sample
from polarization.function import integrate_intensity, sub_intensity
from polarization.io import mute_jax_warnings, perform_cached_doit
from polarization.lhcb import ModelParameters, convert_dict_keys, load_three_body_decays

mute_jax_warnings()

reference_subsystem = 1
dynamics_configurator = load_three_body_decays("../data/isobars.json")
decay = dynamics_configurator.decay
amplitude_builder = DalitzPlotDecompositionBuilder(decay)
amplitude_builder.dynamics_choices = dynamics_configurator
model = amplitude_builder.formulate(reference_subsystem)

## Function creation

In [ ]:
%%time
polarization_exprs = formulate_polarization(amplitude_builder, reference_subsystem)
unfolded_polarization_exprs = [
    perform_cached_doit(expr.doit().xreplace(model.amplitudes))
    for expr in tqdm(polarization_exprs, desc="Unfolding polarization expressions")
]
unfolded_intensity_expr = perform_cached_doit(model.full_expression)

This time, we do not {ref}`substitute certain parameters with their parameter defaults<polarization:Definition of free parameters>`, but lambdify the full expression, so that parameter values can be set for different models. Note that this makes lambdification slower.

In [ ]:
%%time
polarization_funcs = [
    create_parametrized_function(
        unfolded_polarization_exprs[xyz],
        parameters=model.parameter_defaults,
        backend="jax",
    )
    for xyz in tqdm(range(3))
]
intensity_func = create_parametrized_function(
    unfolded_intensity_expr,
    parameters=model.parameter_defaults,
    backend="jax",
)

## Parameter bootstrapping

In [ ]:
n_bootstraps = 200
model_choice = "Default amplitude model."
allowed_model_titles = [
    "Default amplitude model.",
    "Alternative amplitude model with K^*(892) with free mass and width.",
    "Alternative amplitude model with Lz(1670) with free mass and width.",
    "Alternative amplitude model with Lz(1690) with free mass and width.",
    "Alternative amplitude model with Deltares^{++}(1232) with free mass and width.",
    "Alternative amplitude model with Lz(1600), Deltares(1600)^{++},"
    " Deltares(1700)^{++} with free mass and width.",
    "Alternative amplitude model with K^*(700) with free mass and width.",
    "Alternative amplitude model with K^*(1430) with free width.",
    "Alternative amplitude model with free radial parameter d for the Lc resonance,"
    " indicated as dLc.",
]
model_parameters = ModelParameters(
    "../data/modelparameters.json", decay, allowed_model_titles
)
bootstrap_parameters = convert_dict_keys(
    model_parameters.create_parameter_distribution(model_choice, n_bootstraps, seed=0),
    key_converter=str,
)

In [ ]:
resolution = 200
transformer = create_data_transformer(model)
phsp = generate_meshgrid_sample(decay, resolution)
data = transformer(phsp)
X = data["sigma1"]
Y = data["sigma2"]

In [ ]:
resonances = [chain.resonance.name for chain in decay.chains]
original_parameters = dict(intensity_func.parameters)
bootstrap_intensities = []
bootstrap_fit_fractions = defaultdict(list)
bootstrap_polarizations = []
for i in tqdm(
    range(n_bootstraps),
    desc="Computing polarizations and intensities for parameter combinations",
):
    new_parameters = {k: v[i] for k, v in bootstrap_parameters.items()}
    for func in polarization_funcs + [intensity_func]:
        func.update_parameters(original_parameters)
        func.update_parameters(new_parameters)
    bootstrap_polarizations.append([func(data).real for func in polarization_funcs])
    intensity_array = intensity_func(data)
    bootstrap_intensities.append(intensity_array)
    I_tot = integrate_intensity(intensity_array)
    for resonance_name in resonances:
        res_filter = resonance_name.replace("(", R"\(").replace(")", R"\)")
        I_sub = sub_intensity(intensity_func, data, [res_filter])
        fit_fraction = I_sub / I_tot
        bootstrap_fit_fractions[resonance_name].append(fit_fraction)

In [ ]:
bootstrap_intensities = jnp.array(bootstrap_intensities)
bootstrap_polarizations = jnp.array(bootstrap_polarizations)
bootstrap_polarizations = jnp.swapaxes(bootstrap_polarizations, 0, 1)
bootstrap_polarization_norms = jnp.sqrt(jnp.sum(bootstrap_polarizations**2, axis=0))
bootstrap_fit_fractions = {k: jnp.array(v) for k, v in bootstrap_fit_fractions.items()}

In [ ]:
assert bootstrap_intensities.shape == (n_bootstraps, resolution, resolution)
assert bootstrap_polarizations.shape == (3, n_bootstraps, resolution, resolution)
assert bootstrap_polarization_norms.shape == (n_bootstraps, resolution, resolution)

## Statistical uncertainties

In [ ]:
%config InlineBackend.figure_formats = ['png']

In [ ]:
fig, axes = plt.subplots(
    ncols=4,
    nrows=2,
    figsize=(15, 8),
    gridspec_kw={"width_ratios": [1, 1, 1, 1.2]},
    sharex=True,
    sharey=True,
    tight_layout=True,
)
fig.suptitle(R"Polarization sensitivity $\vec\alpha$ (statistical)")
s1_label = R"$\sigma_1=m^2\left(K\pi\right)$"
s2_label = R"$\sigma_2=m^2\left(pK\right)$"
axes[0, 0].set_ylabel(s2_label)
axes[1, 0].set_ylabel(s2_label)

alpha_mean_over_bootstrap = [
    jnp.mean(bootstrap_polarization_norms, axis=0),
    *jnp.mean(bootstrap_polarizations, axis=1),
]
alpha_std_over_bootstrap = [
    jnp.std(bootstrap_polarization_norms, axis=0),
    *jnp.std(bootstrap_polarizations, axis=1),
]
alpha_mean_over_bootstrap = jnp.array(alpha_mean_over_bootstrap)
alpha_std_over_bootstrap = jnp.array(alpha_std_over_bootstrap)

global_max_std = jnp.nanmax(alpha_std_over_bootstrap)
for i in range(4):
    if i != 0:
        title = Rf"$\alpha_{'xyz'[i-1]}$"
    else:
        title = R"$\left|\vec\alpha\right|$"
    axes[0, i].set_title(title)

    mesh = axes[0, i].pcolormesh(X, Y, alpha_mean_over_bootstrap[i], cmap=cm.RdYlGn_r)
    mesh.set_clim(vmin=-1, vmax=+1)
    if axes[0, i] is axes[0, -1]:
        c_bar = fig.colorbar(mesh, ax=axes[0, i], pad=0.01)
        c_bar.ax.set_ylabel(Rf"$\alpha$ averaged with {n_bootstraps} bootstraps")
        c_bar.ax.set_yticks([-1, 0, +1])
        c_bar.ax.set_yticklabels(["-1", "0", "+1"])

    mesh = axes[1, i].pcolormesh(X, Y, alpha_std_over_bootstrap[i])
    mesh.set_clim(vmin=0, vmax=global_max_std)
    axes[1, i].set_xlabel(s1_label)
    if axes[1, i] is axes[1, -1]:
        c_bar = fig.colorbar(mesh, ax=axes[1, i], pad=0.01)
        c_bar.ax.set_ylabel("standard deviation")
plt.show()

In [ ]:
fig, axes = plt.subplots(
    ncols=4,
    nrows=2,
    figsize=(15, 8),
    gridspec_kw={"width_ratios": [1, 1, 1, 1.2]},
    sharex=True,
    sharey=True,
    tight_layout=True,
)
fig.suptitle(R"$\vec\alpha \cdot I$ distributions (statistical)")
axes[0, 0].set_ylabel(s2_label)
axes[1, 0].set_ylabel(s2_label)

alpha_times_I_mean = [
    jnp.mean(bootstrap_polarization_norms * bootstrap_intensities, axis=0),
    *jnp.mean(bootstrap_polarizations * bootstrap_intensities, axis=1),
]
alpha_times_I_std = [
    jnp.std(bootstrap_polarization_norms * bootstrap_intensities, axis=0),
    *jnp.std(bootstrap_polarizations * bootstrap_intensities, axis=1),
]
alpha_times_I_mean = jnp.array(alpha_times_I_mean)
alpha_times_I_std = jnp.array(alpha_times_I_std)

intensity_mean_over_bootstrap = jnp.mean(bootstrap_intensities, axis=0)
global_max_mean = jnp.nanmax(jnp.abs(alpha_times_I_mean))
global_max_std = jnp.nanmax(alpha_times_I_std / intensity_mean_over_bootstrap)
for i in range(4):
    if i != 0:
        title = Rf"$\alpha_{'xyz'[i-1]}$"
    else:
        title = R"$\left|\vec\alpha\right|$"
    axes[0, i].set_title(title)
    axes[1, i].set_xlabel(s1_label)

    mesh = axes[0, i].pcolormesh(X, Y, alpha_times_I_mean[i], cmap=cm.RdYlGn_r)
    mesh.set_clim(vmin=-global_max_mean, vmax=+global_max_mean)
    if axes[0, i] is axes[0, -1]:
        c_bar = fig.colorbar(mesh, ax=axes[0, i], pad=0.01)
        c_bar.ax.set_ylabel(
            Rf"$\alpha \cdot I$ averaged with {n_bootstraps} bootstraps"
        )

    mesh = axes[1, i].pcolormesh(
        X, Y, alpha_times_I_std[i] / intensity_mean_over_bootstrap
    )
    mesh.set_clim(vmin=0, vmax=global_max_std)
    if axes[1, i] is axes[1, -1]:
        c_bar = fig.colorbar(mesh, ax=axes[1, i], pad=0.01)
        c_bar.ax.set_ylabel("standard deviation / intensity")
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12, 6), sharey=True)
fig.suptitle("Intensity distribution (statistical)")
ax1.set_xlabel(s1_label)
ax2.set_xlabel(s1_label)
ax1.set_ylabel(s2_label)

intensity_std_over_bootstrap = jnp.std(bootstrap_intensities, axis=0)

mesh = ax1.pcolormesh(X, Y, intensity_mean_over_bootstrap, cmap=cm.Reds)
fig.colorbar(mesh, ax=ax1, pad=0.01)
ax1.set_title(f"average of {n_bootstraps} bootstraps")

Z = intensity_std_over_bootstrap / intensity_mean_over_bootstrap
mesh = ax2.pcolormesh(X, Y, Z)
fig.colorbar(mesh, ax=ax2, pad=0.01)
ax2.set_title("standard deviation / intensity")
fig.tight_layout()
plt.show()

### Comparison with nominal values

In [ ]:
nominal_parameters = convert_dict_keys(
    model_parameters.get_parameter_values(model_choice),
    key_converter=str,
)
for func in polarization_funcs + [intensity_func]:
    func.update_parameters(original_parameters)
    func.update_parameters(nominal_parameters)
nominal_intensities = intensity_func(data)
nominal_polarizations = jnp.array([func(data).real for func in polarization_funcs])
nominal_polarization_norms = jnp.sqrt(jnp.sum(nominal_polarizations**2, axis=0))

In [ ]:
fig, axes = plt.subplots(
    ncols=4,
    figsize=(15, 4.5),
    gridspec_kw={"width_ratios": [1, 1, 1, 1.2]},
    sharey=True,
    tight_layout=True,
)
fig.suptitle("Comparison with nominal values")
axes[0].set_ylabel(s2_label)

vmax = 5.0  # %
for i in range(4):
    if i != 0:
        title = Rf"$\alpha_{'xyz'[i-1]}$"
        Z = jnp.abs(
            (alpha_mean_over_bootstrap[i] - nominal_polarizations[i - 1])
            / nominal_polarizations[i - 1]
        )
    else:
        title = "$I$"
        Z = 100 * jnp.abs(
            (intensity_mean_over_bootstrap - nominal_intensities) / nominal_intensities
        )
    axes[i].set_title(title)

    mesh = axes[i].pcolormesh(X, Y, Z, cmap=cm.Reds)
    mesh.set_clim(vmin=0, vmax=vmax)
    if axes[i] is axes[-1]:
        c_bar = fig.colorbar(mesh, ax=axes[i], pad=0.01)
        c_bar.ax.set_ylabel("difference with nominal value (%)")

plt.show()

## Systematic uncertainties

In [ ]:
systematic_intensities = []
systematic_polarizations = []
systematic_fit_fractions = defaultdict(list)
for title in tqdm(allowed_model_titles):
    new_parameters = convert_dict_keys(
        model_parameters.get_parameter_values(title),
        key_converter=str,
    )
    for func in polarization_funcs + [intensity_func]:
        func.update_parameters(original_parameters)
        func.update_parameters(new_parameters)
    systematic_polarizations.append([func(data).real for func in polarization_funcs])
    systematic_intensities.append(intensity_func(data))
    I_tot = integrate_intensity(intensity_array)
    for resonance_name in resonances:
        res_filter = resonance_name.replace("(", R"\(").replace(")", R"\)")
        I_sub = sub_intensity(intensity_func, data, [res_filter])
        fit_fraction = I_sub / I_tot
        systematic_fit_fractions[resonance_name].append(fit_fraction)

systematic_intensities = jnp.array(systematic_intensities)
systematic_polarizations = jnp.array(systematic_polarizations)
systematic_polarizations = jnp.swapaxes(systematic_polarizations, 0, 1)
systematic_polarization_norms = jnp.sqrt(jnp.sum(systematic_polarizations**2, axis=0))
systematic_fit_fractions = {
    k: jnp.array(v) for k, v in systematic_fit_fractions.items()
}

In [ ]:
n_models = len(allowed_model_titles)
assert systematic_intensities.shape == (n_models, resolution, resolution)
assert systematic_polarizations.shape == (3, n_models, resolution, resolution)
assert systematic_polarization_norms.shape == (n_models, resolution, resolution)
n_models

In [ ]:
fig, axes = plt.subplots(
    ncols=4,
    nrows=2,
    figsize=(15, 8),
    gridspec_kw={"width_ratios": [1, 1, 1, 1.2]},
    sharex=True,
    sharey=True,
    tight_layout=True,
)
fig.suptitle(R"Polarization sensitivity $\vec\alpha$ (systematics)")
axes[0, 0].set_ylabel(s2_label)
axes[1, 0].set_ylabel(s2_label)

alpha_mean_over_models = [
    jnp.mean(systematic_polarization_norms, axis=0),
    *jnp.mean(systematic_polarizations, axis=1),
]
alpha_std_over_models = [
    jnp.std(systematic_polarization_norms, axis=0),
    *jnp.std(systematic_polarizations, axis=1),
]
alpha_mean_over_models = jnp.array(alpha_mean_over_models)
alpha_std_over_models = jnp.array(alpha_std_over_models)

global_max_std = jnp.nanmax(alpha_std_over_models)
for i in range(4):
    if i != 0:
        title = Rf"$\alpha_{'xyz'[i-1]}$"
    else:
        title = R"$\left|\vec\alpha\right|$"
    axes[0, i].set_title(title)

    mesh = axes[0, i].pcolormesh(X, Y, alpha_mean_over_models[i], cmap=cm.RdYlGn_r)
    mesh.set_clim(vmin=-1, vmax=+1)
    if axes[0, i] is axes[0, -1]:
        c_bar = fig.colorbar(mesh, ax=axes[0, i], pad=0.01)
        c_bar.ax.set_ylabel(Rf"$\alpha$ averaged with {n_models} models")
        c_bar.ax.set_yticks([-1, 0, +1])
        c_bar.ax.set_yticklabels(["-1", "0", "+1"])

    mesh = axes[1, i].pcolormesh(X, Y, alpha_std_over_models[i])
    mesh.set_clim(vmin=0, vmax=global_max_std)
    axes[1, i].set_xlabel(s1_label)
    if axes[1, i] is axes[1, -1]:
        c_bar = fig.colorbar(mesh, ax=axes[1, i], pad=0.01)
        c_bar.ax.set_ylabel("standard deviation")
plt.show()

In [ ]:
fig, axes = plt.subplots(
    ncols=4,
    nrows=2,
    figsize=(15, 8),
    gridspec_kw={"width_ratios": [1, 1, 1, 1.2]},
    sharex=True,
    sharey=True,
    tight_layout=True,
)
fig.suptitle(R"$\vec\alpha \cdot I$ distributions (systematics)")
axes[0, 0].set_ylabel(s2_label)
axes[1, 0].set_ylabel(s2_label)

sys_alpha_times_I_mean = [
    jnp.mean(systematic_polarization_norms * systematic_intensities, axis=0),
    *jnp.mean(systematic_polarizations * systematic_intensities, axis=1),
]
sys_alpha_times_I_std = [
    jnp.std(systematic_polarization_norms * systematic_intensities, axis=0),
    *jnp.std(systematic_polarizations * systematic_intensities, axis=1),
]
sys_alpha_times_I_mean = jnp.array(sys_alpha_times_I_mean)
sys_alpha_times_I_std = jnp.array(sys_alpha_times_I_std)

intensity_mean_over_models = jnp.mean(systematic_intensities, axis=0)
global_max_mean = jnp.nanmax(jnp.abs(sys_alpha_times_I_mean))
global_max_std = jnp.nanmax(sys_alpha_times_I_std / intensity_mean_over_models)
for i in range(4):
    if i != 0:
        title = Rf"$\alpha_{'xyz'[i-1]}$"
    else:
        title = R"$\left|\vec\alpha\right|$"
    axes[0, i].set_title(title)
    axes[1, i].set_xlabel(s1_label)

    mesh = axes[0, i].pcolormesh(X, Y, sys_alpha_times_I_mean[i], cmap=cm.RdYlGn_r)
    mesh.set_clim(vmin=-global_max_mean, vmax=+global_max_mean)
    if axes[0, i] is axes[0, -1]:
        c_bar = fig.colorbar(mesh, ax=axes[0, i], pad=0.01)
        c_bar.ax.set_ylabel(Rf"$\alpha \cdot I$ averaged with {n_models} models")

    mesh = axes[1, i].pcolormesh(
        X, Y, sys_alpha_times_I_std[i] / intensity_mean_over_models
    )
    mesh.set_clim(vmin=0, vmax=global_max_std)
    if axes[1, i] is axes[1, -1]:
        c_bar = fig.colorbar(mesh, ax=axes[1, i], pad=0.01)
        c_bar.ax.set_ylabel("standard deviation / intensity")
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12, 6), sharey=True)
fig.suptitle("Intensity distribution (systematics)")
ax1.set_xlabel(s1_label)
ax2.set_xlabel(s1_label)
ax1.set_ylabel(s2_label)

intensity_std_over_models = jnp.std(systematic_intensities, axis=0)

mesh = ax1.pcolormesh(X, Y, intensity_mean_over_models, cmap=cm.Reds)
fig.colorbar(mesh, ax=ax1, pad=0.01)
ax1.set_title(f"average of {n_models} models")

Z = intensity_std_over_models / intensity_mean_over_models
mesh = ax2.pcolormesh(X, Y, Z)
fig.colorbar(mesh, ax=ax2, pad=0.01)
ax2.set_title("standard deviation / intensity")
fig.tight_layout()
plt.show()

## Fit fractions

In [ ]:
src = r"""
| Resonance | Fit Fraction (%) |
|:----------|:----------------:|
"""
for resonance_name in resonances:
    ff_models = 100 * systematic_fit_fractions[resonance_name]
    ff_bootstraps = 100 * bootstrap_fit_fractions[resonance_name]
    ff_nominal = f"{ff_models[0]:.2f}"
    ff_stat = f"{ff_bootstraps.std():.2f}"
    ff_syst_min = f"{(ff_models[1:]-ff_models[0]).min():+.2f}"
    ff_syst_max = f"{(ff_models[1:]-ff_models[0]).max():+.2f}"
    resonance_latex = resonance_name
    resonance_latex = resonance_latex.replace("L", R"\Lambda")
    resonance_latex = resonance_latex.replace("D", R"\Delta")
    src += f"| ${resonance_latex}$ | "
    src += Rf"${ff_nominal} \pm {ff_stat}_{{{ff_syst_min}}}^{{{ff_syst_max}}}$ |" "\n"
Markdown(src)

## Average polarizations

In [ ]:
def compute_weighted_average(v: jnp.ndarray, weights: jnp.ndarray) -> jnp.ndarray:
    return jnp.nansum(v * weights, axis=(-1, -2)) / jnp.nansum(weights, axis=(-1, -2))


nominal_alpha_norm = compute_weighted_average(
    nominal_polarization_norms,
    weights=nominal_intensities,
)
nominal_alpha = compute_weighted_average(
    nominal_polarizations,
    weights=nominal_intensities,
)

alpha_norm_per_bootstrap = compute_weighted_average(
    bootstrap_polarization_norms,
    weights=bootstrap_intensities,
)
alpha_per_bootstrap = compute_weighted_average(
    bootstrap_polarizations,
    weights=bootstrap_intensities,
)

alpha_norm_per_model = compute_weighted_average(
    systematic_polarization_norms,
    weights=systematic_intensities,
)
alpha_per_model = compute_weighted_average(
    systematic_polarizations,
    weights=systematic_intensities,
)

In [ ]:
alpha_norm_std = alpha_norm_per_bootstrap.std()
alpha_std = alpha_per_bootstrap.std(axis=1)

alpha_norm_sys_diff = alpha_norm_per_model - nominal_alpha_norm
alpha_sys_diff = jnp.subtract(alpha_per_model.T, nominal_alpha)
assert alpha_norm_sys_diff.shape == (n_models,)
assert alpha_sys_diff.shape == (n_models, 3)
alpha_norm_sys_min = alpha_norm_sys_diff.min()
alpha_norm_sys_max = alpha_norm_sys_diff.max()
alpha_sys_min = alpha_sys_diff.min(axis=0)
alpha_sys_max = alpha_sys_diff.max(axis=0)

In [ ]:
def render_uncertainties(value, stat, sys_min, sys_max, plus: bool = True) -> str:
    if plus:
        val = f"{1e3*value:+.1f}"
    else:
        val = f"{1e3*value:.1f}"
    stat = f"{1e3*stat:.1f}"
    sys_min = f"-{1e3*abs(sys_min):.1f}"
    sys_max = f"+{1e3*abs(sys_max):.1f}"
    return (
        Rf"\left({val} \pm {stat}_{{{sys_min}}}^{{{sys_max}}} \right) \times 10^{{-3}}"
    )


src = Rf"""
\begin{{array}}{{ccr}}
  \overline{{\alpha_x}} & = & {render_uncertainties(nominal_alpha[0], alpha_std[0], alpha_sys_min[0], alpha_sys_max[0])} \\
  \overline{{\alpha_y}} & = & {render_uncertainties(nominal_alpha[1], alpha_std[1], alpha_sys_min[1], alpha_sys_max[1])} \\
  \overline{{\alpha_z}} & = & {render_uncertainties(nominal_alpha[2], alpha_std[2], alpha_sys_min[2], alpha_sys_max[2])} \\
  \overline{{\left|\vec{{\alpha}}\right|}} & = & {render_uncertainties(nominal_alpha_norm, alpha_norm_std, alpha_norm_sys_min, alpha_norm_sys_max, False)} \\
\end{{array}}
"""
Latex(src)

Difference between each model and nominal value (computed with the default model):

In [ ]:
pd.DataFrame(
    {
        title.strip("."): (f"{x:+.4f}", f"{y:+.4f}", f"{z:+.4f}", f"{norm:+.4f}")
        for title, (x, y, z), norm in zip(
            allowed_model_titles, alpha_sys_diff, alpha_norm_sys_diff
        )
    }
).transpose().rename(columns={0: "ɑx", 1: "ɑy", 2: "ɑz", 3: "|ɑ|"})